In [83]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm import tqdm
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from collections import OrderedDict
import matplotlib.pyplot as plt
from Network import *
from DataSet import *
from train_procedure import *
from Aug import *
from torchvision import transforms
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
path_test_image = 'data/test_images'
path_model = 'trained_model/best_model3.pth'

In [101]:
dict_train ={"save_model": True,
             'num_model': 2,
            "cross_val": False,
            "skip_connection": True,
            "num_epochs": 400,
            "n_splits": 1,
            "batch_size": 10,
            "scale_channel": 2,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": 0.0,
                "use_dropout": False,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": 1e-05,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "ReduceLROnPlateau",
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau,
            "param_scheduler": {"mode": 'max',
                               "factor": 1./np.sqrt(10),
                               'min_lr': 1e-07,
                               'patience': 2,
                               'verbose': True},
             "scaler": torch.cuda.amp.GradScaler(),
             "device":0
            }

model = UNET({"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": 0.0,
                "use_dropout": False,
                "bias": False}, {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False}
             ,in_channels=3, out_channels=1,init=True,scale_channel=2)

In [102]:
model.load_state_dict(torch.load(path_model,map_location=torch.device('cpu')))
model.eval()

UNET(
  (ups): ModuleList(
    (0): Sequential(
      (trans_conv): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    )
    (1): DoubleConv(
      (conv): Sequential(
        (conv1): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (BatchNorm1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation1): ELU(alpha=1.0, inplace=True)
        (Dropout1): Dropout(p=0.0, inplace=False)
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (BatchNorm2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation2): ELU(alpha=1.0, inplace=True)
        (Dropout2): Dropout(p=0.0, inplace=False)
      )
    )
    (2): Sequential(
      (trans_conv): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): DoubleConv(
      (conv): Sequential(
        (conv1): Conv2d(512, 256, kernel_si

In [103]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
test_dataset = RoadDataset(
        image_dir=path_test_image,
        mask_dir=None,
        transform=test_transform,
    )

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False,
                                    pin_memory=True, num_workers=0)

In [104]:
for idx in tqdm(range(50)):
    #data = torch.unsqueeze(torch.permute(torch.from_numpy(np.array(Image.open(os.path.join(path_test_image,
     #                                                 f'test_{idx+1}.png')).convert("RGB"))),(2,0,1)),0)
    data = transforms.ToTensor()(np.array(Image.open(os.path.join(path_test_image,
                                                      f'test_{idx+1}.png')).convert("RGB"))).unsqueeze(0)
    #print(data.shape)
    output = model(data)
    preds = torch.sigmoid(output)
    preds = (preds > 0.5).float()
    torchvision.utils.save_image(
        torch.Tensor(preds), f"predict_mask/pred_{idx+1}.png"
    )
    #plt.imshow(preds[0,0,:,:].detach().numpy())
    #plt.show()

100%|████████████████████████████████████████| 50/50 [2:23:40<00:00, 172.41s/it]
